In [1]:
# Cargamos las librerias necesarias
import pandas as pd
import numpy as np
from pathlib import Path

## 1. Carga Datos

In [ ]:

# Directorio donde está el notebook
BASE_DIR = Path.cwd()
# Carpeta padre (un nivel arriba)
PARENT_DIR = BASE_DIR.parent

# Directorios de datos
directorio_17 = PARENT_DIR / "17. Set Eléctrico"
directorio_16 = PARENT_DIR / "16. Clima Nacional Ponderado"
directorio_10 = PARENT_DIR / "10. Precio Gas"
directorio_15 = PARENT_DIR / "15. Festividades"
directorio_9 = PARENT_DIR / "09. IPC"


# Carga de datos
df_set_electrico_d = pd.read_csv(f"{directorio_17}/SET_ELECTRICO_D.csv", sep=',', encoding='latin-1')
df_set_clima_nacional_d = pd.read_csv(f"{directorio_16}/CLIMA_NACIONAL_POND.csv", sep=',', encoding='latin-1')
df_precio_gas_tratado = pd.read_csv(f"{directorio_10}/P_GAS_TRATADO.csv", sep=',', encoding='latin-1')
df_ipc_d = pd.read_csv(f"{directorio_9}/IPC_daily.csv", sep=',', encoding='latin-1')
df_festividad= pd.read_csv(f"{directorio_15}/festivos_españa_2019_2023.csv", sep=',', encoding='latin-1')


## Tratamiento de Festividad y merging

In [4]:
# Tratamiento de FESTIVIDADES
df_festividad.rename(columns={df_festividad.columns[0]: 'fecha'}, inplace=True)
df_festividad['fecha'] = pd.to_datetime(df_festividad['fecha'], format='%Y-%m-%d')
df_festividad['Year'] = df_festividad['fecha'].dt.year
df_festividad['Month'] = df_festividad['fecha'].dt.month
df_festividad['Day'] = df_festividad['fecha'].dt.day
df_festividad.drop(df_festividad.columns[:3], axis=1, inplace=True)
# Quitams variable categórica
df_festividad.drop('dia_semana', axis=1, inplace=True)


# MERGE TODOS LOS DATOS
df_total= (df_set_electrico_d
.merge (df_set_clima_nacional_d, on=['Year','Month','Day','DayOfYear'], how='left')
.merge( df_precio_gas_tratado, on=['Year','Month','Day','DayOfYear'], how='left')
.merge( df_festividad, on=['Year','Month','Day'], how='left')
.merge( df_ipc_d, on=['Year','Month','Day'], how='left')
)

# CAmbiamos algunos nombres para mayor claridad
df_total.rename(columns={'Price_mean_EUR_MWh': 'Price_Elec_EUR_MWh',
                            'precios_de_los_productos_mibgas': 'Price_Gas_EUR_MWh'}, inplace=True)

# Guardar resultados
df_total.to_csv(f"{BASE_DIR}/SET_INPUTS_BASE_2.csv", index=False)

# Guardamos el SET_INPUTS_BASE_0 sin las columnas de festivos e IPC
df_sinfestividad=df_total.drop(columns=['festivo','IPC Diario' ], axis=1)
df_sinfestividad.to_csv(f"{BASE_DIR}/SET_INPUTS_BASE_0.csv", index=False)